In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
263,AFG,Asia,Afghanistan,2020-09-19,38919.0,47.0,44.714,1437.0,1.0,2.429,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
458,ALB,Europe,Albania,2020-09-19,12073.0,125.0,150.286,353.0,5.0,3.714,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
722,DZA,Africa,Algeria,2020-09-19,49413.0,219.0,237.286,1659.0,5.0,8.571,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-18,France,424392.0,0.0,31148.0,0.0,0.0,0.0
0,2020-09-18,China,90321.0,0.0,4741.0,0.0,0.0,0.0
0,2020-09-18,Italy,294307.0,0.0,35687.0,0.0,0.0,0.0
0,2020-09-18,Spain,635021.0,0.0,30604.0,0.0,0.0,0.0
0,2020-09-18,United States,6750235.0,0.0,198759.0,0.0,0.0,0.0
0,2020-09-18,World,30693158.0,0.0,955367.0,0.0,0.0,0.0
0,2020-09-18,United Kingdom,384690.0,0.0,41753.0,0.0,0.0,0.0
0,2020-09-18,Germany,269651.0,0.0,9386.0,0.0,0.0,0.0
0,2020-09-18,Iran,416354.0,0.0,24014.0,0.0,0.0,0.0
0,2020-09-18,Turkey,299416.0,0.0,7365.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-19,France,424392.0,428696.0,31148.0,31249.0,-4304.0,-101.0
0,2020-09-19,China,90321.0,90308.0,4741.0,4737.0,13.0,4.0
0,2020-09-19,Italy,294307.0,294932.0,35687.0,35668.0,-625.0,19.0
0,2020-09-18,Spain,635021.0,640040.0,30604.0,30495.0,-5019.0,109.0
0,2020-09-19,United States,6750235.0,6724667.0,198759.0,198589.0,25568.0,170.0
0,2020-09-19,World,30693158.0,30540446.0,955367.0,952724.0,152712.0,2643.0
0,2020-09-19,United Kingdom,384690.0,385936.0,41753.0,41732.0,-1246.0,21.0
0,2020-09-19,Germany,269651.0,270070.0,9386.0,9384.0,-419.0,2.0
0,2020-09-19,Iran,416354.0,416198.0,24014.0,23952.0,156.0,62.0
0,2020-09-19,Turkey,299416.0,299810.0,7365.0,7377.0,-394.0,-12.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")